In [ ]:
# Collection of functions I've written, primarily for plotting
import pyvisa                    # Instrument communication
import time                      # Handle time-related tasks (e.g., delays)
import matplotlib.pyplot as plt  # Plotting graphs and visualizing data
import numpy as np               # Numerical operations, particularly with arrays
import pandas as pd              # Data manipulation and analysis
import os                        # Interact with the operating system, such as handling file paths
import csv                       # Read from and write to CSV files
from datetime import date        # Read current date
from collections import defaultdict

In [ ]:
# Apply default parameters that are shared by all plots. Any property can be changed afterwards if needed.
def start_plot(title, xlabel, ylabel, cm_num=13):
    # Create a color map for the plots
    cm=plt.get_cmap('gist_rainbow')

    fig, plot = plt.subplots(layout='constrained')
    plot.set_title(title, fontdict={'fontsize': 16, 'fontweight': 'bold'}, y = 1.03)
    plot.set_xlabel(xlabel, fontdict={'fontsize': 12})
    plot.set_ylabel(ylabel, fontdict={'fontsize': 12})
    plot.tick_params(axis='both', which='major', labelsize=10)
    plot.set_prop_cycle('color', [cm(1.*i/cm_num) for i in range(cm_num)]) # 13 is the number of temps we've been testing at
    plot.grid()
    return fig, plot

# In finish_plot(): plot, fig, directory, and filename are POSITIONAL arguments - when calling function, must insert those first and in
# the same order. 'legend', 'close', and 'save' are KEYWORD arguments - when calling function, their order does not matter (as long
# as they all come after the positional), but you must write out the name and '=' for each. Keyword arguments are optional. 
# If not provided, the default values in the definition below (True) will be assumed.

# This all the code that needs to run after a plot is filled. 
def finish_plot(fig, plot, directory, filename, legend=True, close=True, save=True): 
    if (legend):
        for line in plot.legend(bbox_to_anchor=(1.02, 1), loc='upper left').get_lines(): # Creates legend
            line.set_linewidth(2.5) # Changes widths of the legend's example lines, NOT the actual lines on the plot
    if (close):
        plt.close(fig) #Saves memory, faster
    if (save):
        fig.savefig(os.path.join(directory, filename))


In [ ]:
# Determine the appropriate scaling for generated plots in Engineering Notation
def scale(list):
    array = np.asarray(list, dtype='float') # Python's lists don't support multiplication by a float. Numpy's arrays do.
    if np.max(np.abs(array)) < 1e-12:
        print("Not implemented")
        return -1
    elif np.max(np.abs(array)) < 1e-9:
        k = 1e12
    elif np.max(np.abs(array)) < 1e-6:
        k = 1e9
    elif np.max(np.abs(array)) < 1e-3:
        k = 1e6
    elif np.max(np.abs(array)) < 1:
        k = 1e3
    elif np.max(np.abs(array)) < 1e3:
        k = 1
    else:
        print("Not implemented")
        return -1
    
    return k*array # Return the scaled array

# Determine the appropriate prefix for arrays that use scale(). 
# PASS THE ORIGINAL LIST, NOT THE SCALED
def prefix(list):
    array = np.asarray(list, dtype='float')
    if np.max(np.abs(array)) < 1e-12:
        print("Not implemented")
        return -1
    elif np.max(np.abs(array)) < 1e-9:
        k = 1e12
        prefix = 'p'
    elif np.max(np.abs(array)) < 1e-6:
        k = 1e9
        prefix = 'n'
    elif np.max(np.abs(array)) < 1e-3:
        k = 1e6
        prefix = 'µ'
    elif np.max(np.abs(array)) < 1:
        k = 1e3
        prefix = 'm'
    elif np.max(np.abs(array)) < 1e3:
        k = 1
    else:
        print("Not implemented")
        return -1

    return prefix

In [1]:
# Determines and returns the highest precision of all elements in a list
# Accepts ints, floats, or string equivalents
def determine_precision(list):
    list = [round(float(element), 13) for element in list] # Round to remove float imprecision (occurs around 16th digit for doubles)
    precisions = []
    for element in list:
        element_str = str(element).rstrip('0') # Removes trailing zeros
        if '.' in element_str:
            precisions.append(len(element_str.split('.')[1])) # Number of digits after '.'
        else:
            precisions.append(0) 
    return max(precisions)

# Determines the highest precision of all elements in a list, returns specified element as a string with that precision. Adds trailing zeros.
# This should avoid poorly sorted or inconsistent file names.
def set_precision(imprecise_element, list):
    imprecise_element = float(imprecise_element)
    return f"{imprecise_element:.{determine_precision(list)}f}"

# Testing set_precision()
print(set_precision(2, [2, 2.21, 3.000]))
print(set_precision(2.21, [2, 2.21, 3.000]))
print(set_precision(3.000, [2, 2.21, 3.000]))

print(f"\n{set_precision(3, [2, 1, 3.0])}") # Works for ints
print(set_precision('3', ['2', '1', '3.0'])) # Works for strings
print(set_precision('2.21', ['2', '2.21', '3.000'])) # Works for strings

2.00
2.21
3.00

3
3
2.21


In [ ]:
# Generate lists of values from names of files/folders in a directory
def read_constants(dir, constant_name, type="f"):
    listA = []
    for filename in os.listdir(dir):
        print(filename)
        if f"{constant_name}=" in filename:
            str = filename[filename.find(f"{constant_name}=")+len(constant_name)+1:] # Removes everything before and up to the equals sign
            num = re.match(r'[+-]?(([0-9]+\.?[0-9]*)|(\.[0-9]+))([eE][+-]?\d+)?', str) # Pulls out int or float from start of the remaining string
            listA.append(num.group())
    if (type == "f"):
        listA = [float(element) for element in listA]
    elif (type == "i"):
        listA = [int(element) for element in listA]
    listA = list(set(listA))
    listA = sorted(listA)
    return listA

# This should work well for when there's an unknown or changing set of values 